In [ ]:
!pip install kagglehub --quiet


import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda:0


In [ ]:
import kagglehub

dataset_path = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
print("Dataset downloaded to:", dataset_path)


Dataset downloaded to: /kaggle/input/stanford-car-dataset-by-classes-folder


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_tfms = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_dir = os.path.join(dataset_path, "car_data/car_data/train")
test_dir = os.path.join(dataset_path, "car_data/car_data/test")


train_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform=train_tfms)
test_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform=test_tfms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

classes = train_dataset.classes
print("Number of classes:", len(classes))


Number of classes: 196


In [ ]:
def get_model(num_classes=196):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model.to(device)


In [ ]:
def eval_model(model, data_loader):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = accuracy_score(all_labels, all_preds) * 100
    prec = precision_score(all_labels, all_preds, average='macro')
    rec = recall_score(all_labels, all_preds, average='macro')

    return acc, prec, rec


In [ ]:
def train_model(model, criterion, optimizer, scheduler, n_epochs=50, patience=5):
    best_acc = 0
    patience_counter = 0
    history = {'train_loss': [], 'train_acc': [], 'val_acc': [], 'val_precision': [], 'val_recall': []}

    for epoch in range(n_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total * 100
        val_acc, val_prec, val_rec = eval_model(model, test_loader)

        print(f"Epoch {epoch+1}/{n_epochs} | Loss: {running_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}% | Precision: {val_prec:.2f} | Recall: {val_rec:.2f}")


        history['train_loss'].append(running_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_rec)

        scheduler.step(val_acc)


        if val_acc > best_acc:
            best_acc = val_acc
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model_state)
    return model, history


In [ ]:
# Initialize model, criterion, optimizer, scheduler
model = get_model(num_classes=len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

model, history = train_model(model, criterion, optimizer, scheduler, n_epochs=50, patience=5)

model_path = "best_car_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to: {model_path}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 73.0MB/s]


Epoch 1/50 | Loss: 1126.3036 | Train Acc: 12.46% | Val Acc: 29.72% | Precision: 0.37 | Recall: 0.29


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/50 | Loss: 625.7504 | Train Acc: 48.44% | Val Acc: 64.00% | Precision: 0.70 | Recall: 0.63


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 5/50 | Loss: 119.9413 | Train Acc: 91.43% | Val Acc: 83.58% | Precision: 0.86 | Recall: 0.84


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 6/50 | Loss: 79.8807 | Train Acc: 94.35% | Val Acc: 85.77% | Precision: 0.88 | Recall: 0.86
Epoch 7/50 | Loss: 56.5412 | Train Acc: 96.18% | Val Acc: 87.51% | Precision: 0.89 | Recall: 0.88


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 8/50 | Loss: 41.7646 | Train Acc: 96.91% | Val Acc: 88.19% | Precision: 0.89 | Recall: 0.88
Epoch 9/50 | Loss: 33.8120 | Train Acc: 97.59% | Val Acc: 87.68% | Precision: 0.90 | Recall: 0.88
Epoch 10/50 | Loss: 30.4349 | Train Acc: 97.67% | Val Acc: 89.07% | Precision: 0.90 | Recall: 0.89
Epoch 11/50 | Loss: 25.6094 | Train Acc: 98.05% | Val Acc: 88.11% | Precision: 0.90 | Recall: 0.88
Epoch 12/50 | Loss: 21.9559 | Train Acc: 98.32% | Val Acc: 89.49% | Precision: 0.90 | Recall: 0.89
Epoch 13/50 | Loss: 23.4769 | Train Acc: 98.08% | Val Acc: 85.30% | Precision: 0.88 | Recall: 0.85
Epoch 14/50 | Loss: 25.9982 | Train Acc: 98.12% | Val Acc: 86.30% | Precision: 0.88 | Recall: 0.86
Epoch 15/50 | Loss: 21.8371 | Train Acc: 98.16% | Val Acc: 86.99% | Precision: 0.88 | Recall: 0.87
Epoch 16/50 | Loss: 18.6948 | Train Acc: 98.44% | Val Acc: 87.12% | Precision: 0.89 | Recall: 0.87
Epoch 17/50 | Loss: 8.0040 | Train Acc: 99.51% | Val Acc: 91.67% | Precision: 0.92 | Recall: 0.92
Epoch 18/50 |

In [ ]:
from PIL import Image
import torch.nn.functional as F

def predict_image(image_path, model, transform, class_names):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
        confidence, pred_class = torch.max(probs, 1)

    predicted_label = class_names[pred_class.item()]
    confidence_percent = confidence.item() * 100

    print(f"Prediction: {predicted_label} | Confidence: {confidence_percent:.2f}%")


In [ ]:
image_paths = [
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/AM General Hummer SUV 2000/00076.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Acura ZDX Hatchback 2012/00036.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Jaguar XK XKR 2012/00525.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Dodge Dakota Club Cab 2007/01462.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Mercedes-Benz SL-Class Coupe 2009/01773.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Toyota 4Runner SUV 2012/01272.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Tesla Model S Sedan 2012/04710.jpg",
    "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Porsche Panamera Sedan 2012/01062.jpg",
    "/content/pr pa.jpg"
]

for path in image_paths:
    predict_image(path, model, test_tfms, classes)


Prediction: AM General Hummer SUV 2000 | Confidence: 99.95%
Prediction: Acura ZDX Hatchback 2012 | Confidence: 100.00%
Prediction: Jaguar XK XKR 2012 | Confidence: 99.31%
Prediction: Dodge Dakota Club Cab 2007 | Confidence: 99.91%
Prediction: Mercedes-Benz SL-Class Coupe 2009 | Confidence: 95.62%
Prediction: Toyota 4Runner SUV 2012 | Confidence: 99.95%
Prediction: Tesla Model S Sedan 2012 | Confidence: 99.02%
Prediction: Porsche Panamera Sedan 2012 | Confidence: 99.98%
Prediction: Porsche Panamera Sedan 2012 | Confidence: 99.38%
